In [9]:
import sys
sys.path.append("/planning_through_contact")

In [10]:
import numpy as np
import pandas as pd
import os
from examples.planar_hand.contact_sampler import PlanarHandContactSampler

from qsim.parser import QuasistaticParser

from irs_rrt.irs_rrt import IrsNode
from irs_rrt.irs_rrt_projection import IrsRrtProjection
from irs_rrt.rrt_params import IrsRrtProjectionParams, SmoothingMode
from examples.planar_hand.planar_hand_setup import *

from irs_mpc2.quasistatic_visualizer import QuasistaticVisualizer


In [11]:

# %% quasistatic system
q_parser = QuasistaticParser(q_model_path)

q_vis = QuasistaticVisualizer.make_visualizer(q_parser)
q_sim, q_sim_py = q_vis.q_sim, q_vis.q_sim_py
plant = q_sim.get_plant()

dim_x = q_sim.num_dofs()
dim_u = q_sim.num_actuated_dofs()
idx_a_l = plant.GetModelInstanceByName(robot_l_name)
idx_a_r = plant.GetModelInstanceByName(robot_r_name)
idx_u = plant.GetModelInstanceByName(object_name)
contact_sampler = PlanarHandContactSampler(
    q_sim=q_sim, q_sim_py=q_sim_py, pinch_prob=0.1
)

q_u0 = np.array([0.0, 0.35, 0])
q0_dict = contact_sampler.calc_enveloping_grasp(q_u0)
q0 = q_sim.get_q_vec_from_dict(q0_dict)


joint_limits = {
    idx_u: np.array([[-0.3, 0.3], [0.3, 0.5], [-0.01, np.pi + 0.01]])
}

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [12]:

# %% RRT testing
rrt_params = IrsRrtProjectionParams(q_model_path, joint_limits)
rrt_params.h = h
rrt_params.smoothing_mode = SmoothingMode.k1AnalyticPyramid
rrt_params.log_barrier_weight_for_bundling = 100
rrt_params.root_node = IrsNode(q0)
rrt_params.max_size = 100
rrt_params.goal = np.copy(q0)
rrt_params.goal[2] = np.pi
rrt_params.termination_tolerance = 0.01
rrt_params.goal_as_subgoal_prob = 0.3
rrt_params.regularization = 1e-4
rrt_params.rewire = False
rrt_params.grasp_prob = 0.
rrt_params.distance_threshold = np.inf
rrt_params.stepsize = 0.35
rrt_params.distance_metric = "local_u"

# params.distance_metric = 'global'  # If using global metric
rrt_params.global_metric = np.array([10.0, 10.0, 1.0, 0.1, 0.1, 0.1, 0.1])

print("start state:", q0)
print("goal state:", rrt_params.goal)

start state: [ 0.          0.35        0.         -0.88050497 -0.60592241  0.88050497
  0.60592241]
goal state: [ 0.          0.35        3.14159265 -0.88050497 -0.60592241  0.88050497
  0.60592241]


In [13]:
rrt_params.use_free_solvers = True
# contact_sampler.sim_params.use_free_solvers = True

In [14]:

prob_rrt = IrsRrtProjection(rrt_params, contact_sampler, q_sim, q_sim_py)
prob_rrt.iterate()

 99%|█████████▉| 99/100 [00:01<00:00, 50.18it/s]


In [15]:
(
    q_knots_trimmed,
    u_knots_trimmed,
) = prob_rrt.get_trimmed_q_and_u_knots_to_goal()
q_vis.publish_trajectory(q_knots_trimmed, h=rrt_params.h)

closest distance to subgoal 497.49533558657384


In [16]:
d_batch = prob_rrt.calc_distance_batch(rrt_params.goal)
print("minimum distance: ", d_batch.min())

# %%
node_id_closest = np.argmin(d_batch)

# %%
prob_rrt.save_tree(f"planar_hand_tree_{rrt_params.max_size}_{0}.pkl")


# %%
closest_state = prob_rrt.q_matrix[node_id_closest, :]
print("closest state", closest_state)
print("goal state", rrt_params.goal)


minimum distance:  497.49533558657384
closest state [ 0.          0.35        0.         -0.88050497 -0.60592241  0.88050497
  0.60592241]
goal state [ 0.          0.35        3.14159265 -0.88050497 -0.60592241  0.88050497
  0.60592241]
